In [1]:
import pandas as pd
import numpy as np
import os
import pandas_datareader as pdr
from datetime import datetime

In [147]:
# set time frame
timeframe = '1D'

# start and end date
start_date = datetime(2017,6,1)
end_date = datetime(2021,2,19)

# set tickers
tickers = ['TOT']

#Which ticker from above list would you like to use?
#ticker = 0

# pulling data from pdr
tickers_df = pdr.DataReader(tickers,'yahoo',start_date,end_date)

In [148]:
%store tickers

Stored 'tickers' (list)


In [149]:
tickers_df.head()

Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,TOT,TOT,TOT,TOT,TOT,TOT
Date,,,,,,
2017-06-01,42.105991,52.099998,52.369999,52.049999,52.160000,859500.0
2017-06-02,42.194893,52.209999,52.209999,51.700001,51.970001,1438900.0
2017-06-05,41.831215,51.759998,51.860001,51.430000,51.720001,2950800.0
2017-06-06,41.968609,51.930000,51.939999,51.299999,51.360001,2466400.0
2017-06-07,41.354389,51.169998,51.790001,50.849998,51.630001,1372800.0


In [150]:
# closing prices dataframe
data = pd.DataFrame()

# get closing prices for all tickers in tickers
for ticker in tickers:
    data = pd.DataFrame()
    data[ticker]=tickers_df['Adj Close'][ticker]

In [151]:
data.rename(columns={tickers[0]: 'Close'}, inplace=True)

In [152]:
data

,Close
Date,
2017-06-01,42.105991
2017-06-02,42.194893
2017-06-05,41.831215
2017-06-06,41.968609
2017-06-07,41.354389
...,...
2021-02-12,42.770000
2021-02-16,44.150002
2021-02-17,45.049999


In [153]:
# add percent change
data['daily_return'] = data['Close'].pct_change().dropna()
data['volume'] = tickers_df['Volume'][ticker]

In [154]:
data = data.dropna()
data.head()

,Close,daily_return,volume
Date,,,
2017-06-02,42.194893,0.002111,1438900.0
2017-06-05,41.831215,-0.008619,2950800.0
2017-06-06,41.968609,0.003284,2466400.0
2017-06-07,41.354389,-0.014635,1372800.0
2017-06-08,41.313984,-0.000977,659500.0


In [155]:
# EWMA Short and Long Windows
short_window = 5
long_window = 50

# Fast and Slow EWMAs from short and long windows
data['ewma_short'] = data['Close'].ewm(halflife=short_window).mean()
data['ewma_long'] = data['Close'].ewm(halflife=long_window).mean()

# Construct a crossover trading signal
data['crossover_long'] = np.where(data['ewma_short'] > data['ewma_long'], 1.0, 0.0) 

# making sep columns if (where) fast passes close == long 
data['crossover_short'] = np.where(data['ewma_short'] < data['ewma_long'], -1.0, 0.0)
data['crossover_signal'] = data['crossover_long'] + data['crossover_short']


<ipython-input-155-06876fc3a873>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ewma_short'] = data['Close'].ewm(halflife=short_window).mean()
<ipython-input-155-06876fc3a873>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ewma_long'] = data['Close'].ewm(halflife=long_window).mean()
<ipython-input-155-06876fc3a873>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [156]:
# Volatility - Short and Long Windows
short_vol_window = 5
long_vol_window = 50

# Construct a `Fast` and `Slow` Exponential Moving Average from short and long windows, respectively
data['short_vol'] = data['daily_return'].ewm(halflife=short_vol_window).std()
data['long_vol'] = data['daily_return'].ewm(halflife=long_vol_window).std()

# Construct a crossover trading signal
data['vol_trend_long'] = np.where(data['short_vol'] < data['long_vol'], 1.0, 0.0)
data['vol_trend_short'] = np.where(data['short_vol'] > data['long_vol'], -1.0, 0.0) 
data['vol_trend_signal'] = data['vol_trend_long'] + data['vol_trend_short']


<ipython-input-156-508068c14cbe>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['short_vol'] = data['daily_return'].ewm(halflife=short_vol_window).std()
<ipython-input-156-508068c14cbe>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['long_vol'] = data['daily_return'].ewm(halflife=long_vol_window).std()
<ipython-input-156-508068c14cbe>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [157]:
data = data.dropna()

In [158]:
# Set bollinger band window
bollinger_window = 20

# Calculate rolling mean and standard deviation
data['bollinger_mid_band'] = data['Close'].rolling(window=bollinger_window).mean()
data['bollinger_std'] = data['Close'].rolling(window=20).std() # will use for upper and lower bands of bollinger

# Calculate upper and lowers bands of bollinger band
data['bollinger_upper_band']  = data['bollinger_mid_band'] + (data['bollinger_std'] * 1)
data['bollinger_lower_band']  = data['bollinger_mid_band'] - (data['bollinger_std'] * 1)

# Calculate bollinger band trading signal # signal to calc (doing mean revering strategy)
data['bollinger_long'] = np.where(data['Close'] < data['bollinger_lower_band'], 1.0, 0.0)
data['bollinger_short'] = np.where(data['Close'] > data['bollinger_upper_band'], -1.0, 0.0)
data['bollinger_signal'] = data['bollinger_long'] + data['bollinger_short']

In [159]:
#Calculate simple moving average of volume
data['Short_Volume'] = data['volume'].transform(lambda x: x.rolling(window = 5).mean())
data['Long_Volume'] = data['volume'].transform(lambda x: x.rolling(window = 15).mean())
data['SMA_Volume_Ratio'] = data['Short_Volume']/data['Long_Volume']

# Construct a crossover trading signal
data['crossover_vol_long'] = np.where(data['Short_Volume'] > data['Long_Volume'], 1.0, 0.0) 

# making sep columns if (where) fast passes close == long 
data['crossover_vol_short'] = np.where(data['Short_Volume'] < data['Long_Volume'], -1.0, 0.0)
data['crossover_vol_signal'] = data['crossover_vol_long'] + data['crossover_vol_short']

In [160]:
#Calculate the average true range and keep it in a function
def add_atr_to_dataframe (dataframe, newdf):
    newdf['ATR1'] = abs (dataframe['High'][ticker] - dataframe['Low'][ticker])
    newdf['ATR2'] = abs (dataframe['High'][ticker] - dataframe['Close'][ticker].shift())
    newdf['ATR3'] = abs (dataframe['Low'][ticker] - dataframe['Close'][ticker].shift())
    newdf['TrueRange'] = newdf[['ATR1', 'ATR2', 'ATR3']].max(axis=1)
    return newdf

#Call the average true range function
add_atr_to_dataframe(tickers_df, data)

,Close,daily_return,volume,ewma_short,ewma_long,crossover_long,crossover_short,crossover_signal,short_vol,long_vol,...,Short_Volume,Long_Volume,SMA_Volume_Ratio,crossover_vol_long,crossover_vol_short,crossover_vol_signal,ATR1,ATR2,ATR3,TrueRange
Date,,,,,,,,,,,,,,,,,,,,,
2017-06-05,41.831215,-0.008619,2950800.0,42.000470,42.011794,0.0,-1.0,-1.0,0.007588,0.007588,...,NaN,NaN,NaN,0.0,0.0,0.0,0.430000,0.349998,0.779999,0.779999
2017-06-06,41.968609,0.003284,2466400.0,41.988348,41.997199,0.0,-1.0,-1.0,0.006617,0.006565,...,NaN,NaN,NaN,0.0,0.0,0.0,0.639999,0.180000,0.459999,0.639999
2017-06-07,41.354389,-0.014635,1372800.0,41.795548,41.833139,0.0,-1.0,-1.0,0.008946,0.008671,...,NaN,NaN,NaN,0.0,0.0,0.0,0.940002,0.139999,1.080002,1.080002
2017-06-08,41.313984,-0.000977,659500.0,41.670871,41.726410,0.0,-1.0,-1.0,0.007739,0.007653,...,NaN,NaN,NaN,0.0,0.0,0.0,0.250000,0.040001,0.209999,0.250000
2017-06-09,41.516022,0.004890,1252100.0,41.635376,41.690119,0.0,-1.0,-1.0,0.007848,0.007712,...,1740320.0,NaN,NaN,0.0,0.0,0.0,0.590000,0.360001,0.230000,0.590000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-12,42.770000,0.014950,1810600.0,42.778731,40.052772,1.0,0.0,1.0,0.017288,0.026218,...,3283380.0,2.895940e+06,1.133787,1.0,0.0,1.0,0.959999,0.660000,0.299999,0.959999
2021-02-16,44.150002,0.032266,1869800.0,42.956242,40.109180,1.0,0.0,1.0,0.019625,0.026287,...,2928020.0,2.861113e+06,1.023385,1.0,0.0,1.0,0.680000,1.619999,0.939999,1.619999
2021-02-17,45.049999,0.020385,2623400.0,43.227277,40.177201,1.0,0.0,1.0,0.019147,0.026195,...,2706660.0,2.851933e+06,0.949061,0.0,-1.0,-1.0,1.150002,0.980000,0.170002,1.150002


In [161]:
# Construct a crossover trading signal
data['crossover_atr_entry'] = np.where(data['TrueRange'] < 0.7, 1.0, 0.0) 

# making sep columns if (where) fast passes close == long 
data['crossover_atr_exit'] = np.where(data['TrueRange'] > 0.9, -1.0, 0.0)
data['crossover_atr_signal'] = data['crossover_atr_entry'] + data['crossover_atr_exit']

data

,Close,daily_return,volume,ewma_short,ewma_long,crossover_long,crossover_short,crossover_signal,short_vol,long_vol,...,crossover_vol_long,crossover_vol_short,crossover_vol_signal,ATR1,ATR2,ATR3,TrueRange,crossover_atr_entry,crossover_atr_exit,crossover_atr_signal
Date,,,,,,,,,,,,,,,,,,,,,
2017-06-05,41.831215,-0.008619,2950800.0,42.000470,42.011794,0.0,-1.0,-1.0,0.007588,0.007588,...,0.0,0.0,0.0,0.430000,0.349998,0.779999,0.779999,0.0,0.0,0.0
2017-06-06,41.968609,0.003284,2466400.0,41.988348,41.997199,0.0,-1.0,-1.0,0.006617,0.006565,...,0.0,0.0,0.0,0.639999,0.180000,0.459999,0.639999,1.0,0.0,1.0
2017-06-07,41.354389,-0.014635,1372800.0,41.795548,41.833139,0.0,-1.0,-1.0,0.008946,0.008671,...,0.0,0.0,0.0,0.940002,0.139999,1.080002,1.080002,0.0,-1.0,-1.0
2017-06-08,41.313984,-0.000977,659500.0,41.670871,41.726410,0.0,-1.0,-1.0,0.007739,0.007653,...,0.0,0.0,0.0,0.250000,0.040001,0.209999,0.250000,1.0,0.0,1.0
2017-06-09,41.516022,0.004890,1252100.0,41.635376,41.690119,0.0,-1.0,-1.0,0.007848,0.007712,...,0.0,0.0,0.0,0.590000,0.360001,0.230000,0.590000,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-12,42.770000,0.014950,1810600.0,42.778731,40.052772,1.0,0.0,1.0,0.017288,0.026218,...,1.0,0.0,1.0,0.959999,0.660000,0.299999,0.959999,0.0,-1.0,-1.0
2021-02-16,44.150002,0.032266,1869800.0,42.956242,40.109180,1.0,0.0,1.0,0.019625,0.026287,...,1.0,0.0,1.0,0.680000,1.619999,0.939999,1.619999,0.0,-1.0,-1.0
2021-02-17,45.049999,0.020385,2623400.0,43.227277,40.177201,1.0,0.0,1.0,0.019147,0.026195,...,0.0,-1.0,-1.0,1.150002,0.980000,0.170002,1.150002,0.0,-1.0,-1.0


In [162]:
for ticker in tickers:
    data.to_csv("../Data/" + str(tickers[0]) +  "_Closing_CURRENT" +  ".csv",index=True)